# Российские корпоративные данные: RFSD

## Russian Financial Statements Database (RFSD)

[RFSD](https://github.com/irlcode/RFSD) - это открытая база данных финансовой отчетности российских компаний за 2011-2024 годы.

**Основные характеристики:**
- Первая открытая база данных с информацией о всех активных компаниях России
- Включает компании, которые не подавали отчетность (non-filing firms)
- Источники данных: Росстат и Федеральная налоговая служба
- Покрывает период 2011-2024, обновляется ежегодно
- Данные восстановлены через неинвазивную импутацию, артикуляцию и гармонизацию

**Формат данных:**
- Структурированный формат Apache Parquet с партиционированием по годам
- Неконсолидированная отчетность (форма №1, форма №2, форма №4)
- Позволяет запрашивать только нужные переменные в масштабе

**Документация:**
- GitHub: https://github.com/irlcode/RFSD
- Статья: https://doi.org/10.1038/s41597-025-05150-1


## ОКВЭД: Общероссийский классификатор видов экономической деятельности

**ОКВЭД 2** (Общероссийский классификатор видов экономической деятельности, редакция 2) — это классификатор видов экономической деятельности, используемый в России для кодирования видов деятельности юридических лиц и индивидуальных предпринимателей.

### Что такое ОКВЭД?

ОКВЭД используется для:
- **Статистического учета** — группировки предприятий по видам деятельности
- **Налогообложения** — определения налогового режима
- **Регистрации** — указания видов деятельности при создании компании
- **Анализа** — сравнения компаний из одной отрасли

### Структура кода ОКВЭД:

Код ОКВЭД имеет иерархическую структуру:
- **Секция** (буква A-U) — крупные отрасли экономики
- **Класс** (2 цифры, например `01`) — общая группа видов деятельности
- **Подкласс** (3-4 цифры, например `01.1`) — более детальная группа
- **Группа** (5-6 цифр, например `01.11`) — конкретная группа видов деятельности
- **Подгруппа** (7-8 цифр, например `01.11.1`) — детальная подгруппа
- **Вид** (9-10 цифр, например `01.11.11`) — конкретный вид деятельности

**Пример:** `46.35.1` означает:
- `46` — Торговля оптовая непродовольственными потребительскими товарами
- `46.3` — Торговля оптовая пищевыми продуктами, напитками и табачными изделиями
- `46.35` — Торговля оптовая напитками
- `46.35.1` — Торговля оптовая безалкогольными напитками

### Основной и дополнительные коды ОКВЭД:

При регистрации компании указываются:

1. **Основной код ОКВЭД** — основной вид экономической деятельности компании
   - Определяется по тому виду деятельности, который приносит наибольшую выручку
   - Используется для статистической отчетности и классификации компании

2. **Дополнительные коды ОКВЭД** — другие виды деятельности, которыми занимается компания
   - Может быть несколько дополнительных кодов
   - Компания может заниматься любыми видами деятельности, указанными в кодах

**Важно:** Компания может иметь несколько кодов ОКВЭД, но только один основной. 

В RFSD и других подобных сервисах обычно хранится основной код ОКВЭД компании.

### Секции ОКВЭД 2:

- **A** — Сельское, лесное хозяйство, охота, рыболовство и рыбоводство
- **B** — Добыча полезных ископаемых
- **C** — Обрабатывающие производства
- **D** — Обеспечение электрической энергией, газом и паром; кондиционирование воздуха
- **E** — Водоснабжение; водоотведение, организация сбора и утилизации отходов
- **F** — Строительство
- **G** — Торговля оптовая и розничная
- **H** — Транспортировка и хранение
- **I** — Деятельность гостиниц и предприятий общественного питания
- **J** — Деятельность в области информации и связи
- **K** — Деятельность финансовая и страховая
- **L** — Операции с недвижимым имуществом
- **M** — Деятельность профессиональная, научная и техническая
- **N** — Деятельность административная и сопутствующие дополнительные услуги
- **O** — Государственное управление и обеспечение военной безопасности
- **P** — Образование
- **Q** — Деятельность в области здравоохранения и социальных услуг
- **R** — Деятельность в области культуры, спорта, организации досуга и развлечений
- **S** — Предоставление прочих видов услуг
- **T** — Деятельность домашних хозяйств как работодателей
- **U** — Деятельность экстерриториальных организаций и органов

### Использование ОКВЭД в анализе:

В RFSD переменная `okved` содержит основной код ОКВЭД компании, а `okved_section` — секцию (букву A-U). Это позволяет:
- Группировать компании по отраслям
- Сравнивать финансовые показатели компаний из одной отрасли
- Анализировать отраслевую структуру экономики
- Выявлять отраслевые закономерности

**Пример:** Для анализа нефтегазовых компаний можно использовать секцию `B` (Добыча полезных ископаемых) или более конкретные коды, например `06` (Добыча сырой нефти и природного газа).


### Работа со справочником ОКВЭД

Для расшифровки кодов ОКВЭД можно использовать справочник, который содержит все коды с их названиями и описаниями.


In [16]:
# Загрузка справочника ОКВЭД 2
import json
from pathlib import Path
import polars as pl

OKVED_PATH = Path("../metadata/okved_2.json")

# Загружаем справочник ОКВЭД
with open(OKVED_PATH, 'r', encoding='utf-8') as f:
    okved_dict = json.load(f)

# Конвертируем в Polars DataFrame для удобной работы
okved_df = pl.DataFrame(okved_dict)

print(f"Загружено {len(okved_df):,} кодов ОКВЭД")
print(f"\nСтруктура справочника:")
print(okved_df.head())
print(f"\nСтолбцы: {okved_df.columns}")


Загружено 2,818 кодов ОКВЭД

Структура справочника:
shape: (5, 5)
┌──────────┬─────────────┬─────────┬───────────────────────────┬───────────────────────────┐
│ code     ┆ parent_code ┆ section ┆ name                      ┆ comment                   │
│ ---      ┆ ---         ┆ ---     ┆ ---                       ┆ ---                       │
│ str      ┆ str         ┆ str     ┆ str                       ┆ str                       │
╞══════════╪═════════════╪═════════╪═══════════════════════════╪═══════════════════════════╡
│ 01       ┆ A           ┆ A       ┆ Растениеводство и         ┆ Эта группировка включает: │
│          ┆             ┆         ┆ животноводство, охота и   ┆ - два основных вида       │
│          ┆             ┆         ┆ предоставление            ┆ деятельности, а именно:   │
│          ┆             ┆         ┆ соответствующих услуг в   ┆ производство продукции    │
│          ┆             ┆         ┆ этих областях             ┆ растени…                  │
│ 01

In [17]:
# Функция для расшифровки кода ОКВЭД
def decode_okved(code, okved_data=okved_df):
    """
    Расшифровывает код ОКВЭД, возвращая название и описание
    
    Parameters:
    -----------
    code : str
        Код ОКВЭД (например, "46.35.1" или "01.11")
    okved_data : pl.DataFrame
        DataFrame со справочником ОКВЭД
        
    Returns:
    --------
    dict
        Словарь с информацией о коде ОКВЭД
    """
    result = okved_data.filter(pl.col('code') == code)
    
    if len(result) > 0:
        row = result.row(0, named=True)
        return {
            'code': row['code'],
            'name': row['name'],
            'section': row['section'],
            'parent_code': row.get('parent_code', None),
            'comment': row.get('comment', '')
        }
    else:
        return None

# Примеры использования
examples = ['46.35.1', '01.11', '06.10']
print("Примеры расшифровки кодов ОКВЭД:")
for code in examples:
    decoded = decode_okved(code)
    if decoded:
        print(f"\nКод: {decoded['code']}")
        print(f"Название: {decoded['name']}")
        print(f"Секция: {decoded['section']}")
    else:
        print(f"\nКод {code} не найден в справочнике")


Примеры расшифровки кодов ОКВЭД:

Код 46.35.1 не найден в справочнике

Код: 01.11
Название: Выращивание зерновых (кроме риса), зернобобовых культур и семян масличных культур
Секция: A

Код: 06.10
Название: Добыча нефти и нефтяного (попутного) газа
Секция: B


In [18]:
# Поиск кодов ОКВЭД по ключевым словам
def find_okved_by_keyword(keyword, okved_data=okved_df):
    """
    Находит коды ОКВЭД по ключевому слову в названии
    
    Parameters:
    -----------
    keyword : str
        Ключевое слово для поиска
    okved_data : pl.DataFrame
        DataFrame со справочником ОКВЭД
        
    Returns:
    --------
    pl.DataFrame
        DataFrame с найденными кодами ОКВЭД
    """
    return (
        okved_data
        .filter(pl.col('name').str.to_lowercase().str.contains(keyword.lower()))
        .select(['code', 'name', 'section'])
    )

# Пример: поиск кодов связанных с нефтью
print("Коды ОКВЭД, связанные с 'нефть':")
oil_codes = find_okved_by_keyword('нефт')
print(oil_codes.head(10))


Коды ОКВЭД, связанные с 'нефть':
shape: (10, 3)
┌─────────┬──────────────────────────────────────────────────────────────────────┬─────────┐
│ code    ┆ name                                                                 ┆ section │
│ ---     ┆ ---                                                                  ┆ ---     │
│ str     ┆ str                                                                  ┆ str     │
╞═════════╪══════════════════════════════════════════════════════════════════════╪═════════╡
│ 06      ┆ Добыча нефти и природного газа                                       ┆ B       │
│ 06.1    ┆ Добыча нефти и нефтяного (попутного) газа                            ┆ B       │
│ 06.10   ┆ Добыча нефти и нефтяного (попутного) газа                            ┆ B       │
│ 06.10.1 ┆ Добыча нефти                                                         ┆ B       │
│ 06.10.3 ┆ Добыча нефтяного (попутного) газа                                    ┆ B       │
│ 09.1    ┆ Предоставл

---

## Практическое задание 1: Работа со справочником ОКВЭД

**Время: 5-7 минут**

**Задание:**
1. Используя функцию `decode_okved()`, расшифруйте следующие коды ОКВЭД:
   - `62.01` 
   - `47.11` 
   - `64.20` 

2. Используя функцию `find_okved_by_keyword()`, найдите все коды ОКВЭД, связанные с:
   - "финанс"
   - "строитель"
   - "образован"

3. Определите, к какой секции ОКВЭД относятся найденные коды.



In [19]:
# Ваше решение здесь:
# 1. Расшифровка кодов ОКВЭД
# decode_okved('62.01')
# decode_okved('47.11')
# decode_okved('64.20')

# 2. Поиск по ключевым словам
# find_okved_by_keyword('финанс')
# find_okved_by_keyword('строитель')
# find_okved_by_keyword('образован')


## Формат данных: Parquet vs CSV

RFSD использует формат **Apache Parquet** вместо традиционного CSV. Понимание различий важно для эффективной работы с данными.

### Что такое Parquet?

**Apache Parquet** — это колоночно-ориентированный формат хранения данных, оптимизированный для аналитических запросов и работы с большими данными.

### Преимущества Parquet перед CSV:

1. **Эффективность памяти:**
   - Parquet использует сжатие данных (обычно в 5-10 раз меньше размер файла)
   - CSV хранит все данные в текстовом виде без сжатия
   - Пример: CSV файл 10 GB может занимать всего 1-2 GB в формате Parquet

2. **Скорость чтения:**
   - Parquet позволяет читать только нужные столбцы (column pruning)
   - CSV всегда читает весь файл целиком
   - Для анализа с 10 столбцами из 200 это означает чтение в 20 раз меньше данных

3. **Типы данных:**
   - Parquet сохраняет типы данных (int, float, date, string)
   - CSV хранит все как текст, требуется дальнейший парсинг при чтении
   - Меньше ошибок и быстрее обработка

4. **Метаданные:**
   - Parquet содержит схему данных (schema) в файле
   - CSV требует угадывать или указывать типы данных вручную
   - Упрощает работу с данными

5. **Партиционирование:**
   - Parquet поддерживает эффективное партиционирование (например, по годам)
   - Можно читать только нужные партиции
   - CSV требует чтения всего файла даже для выборки по одному году

6. **Оптимизация запросов:**
   - Parquet поддерживает predicate pushdown (фильтрация на уровне чтения)
   - Фильтры применяются ДО загрузки данных в память
   - CSV требует загрузки всех данных перед фильтрацией

### Когда использовать CSV?

CSV все еще полезен для:
- Небольших файлов (< 100 MB)
- Обмена данными с людьми (читаемость). 
- Простых задач без требований к производительности
- Загрузки в Excel (относительно простой)

### Для RFSD (60+ млн строк):

Parquet критически важен, потому что:
- Полный датасет в CSV занял бы ~50-100 GB
- В Parquet это ~7-10 GB (сжатие)
- Чтение только нужных столбцов ускоряет работу в десятки раз
- Фильтрация на уровне чтения экономит память и время

**Вывод:** Для больших аналитических датасетов Parquet — стандарт индустрии. Современные инструменты (Polars, DuckDB и проч.) оптимизированы для работы с Parquet. В большинстве случаев нет особо важных причин для использования CSV. 


## Pandas vs Polars: когда что использовать?

В этом курсе мы будем использовать **Polars** для работы с большими данными RFSD, но также упоминаем **pandas**. Важно понимать различия и когда использовать каждую библиотеку.

### Что общего?

Обе библиотеки предоставляют:
- DataFrame структуру данных (таблицы с именованными столбцами)
- Похожий API для основных операций (фильтрация, группировка, агрегация)
- Интеграцию с другими библиотеками Python (matplotlib, numpy)
- Поддержку работы с различными форматами данных (CSV, Parquet, Excel)

### Ключевые различия:

| Характеристика | Pandas | Polars |
|----------------|--------|--------|
| **Производительность** | Медленнее для больших данных | В 5-30 раз быстрее |
| **Многопоточность** | Однопоточный по умолчанию | Многопоточный по умолчанию |
| **Память** | Копирует данные при операциях | Более эффективное использование памяти |
| **Lazy Evaluation** | Нет (eager execution) | Да (опционально через LazyFrame) |
| **API** | Более зрелый, больше функций | Более современный, фокусированный |
| **Сообщество** | Огромное, много примеров | Растущее, меньше примеров |
| **Совместимость** | Стандарт индустрии | Новый, но быстро набирает популярность |

### Когда использовать Pandas?

**Pandas подходит для:**
- Небольших и средних датасетов (< 1-2 GB в памяти)
- Когда нужны специфические функции pandas (например, `pivot_table`, `melt`)
- Работы с существующим кодом на pandas
- Интеграции с библиотеками, которые требуют pandas (например, некоторые визуализации)
- Обучения (больше примеров и документации)

**Примеры использования pandas:**
```python
import pandas as pd
df = pd.read_csv('small_data.csv')  # Небольшой файл
df.groupby('region')['revenue'].mean()  # Простые операции
df.plot()  # Встроенная визуализация
```

### Когда использовать Polars?

**Polars подходит для:**
- Больших датасетов (> 1-2 GB, миллионы строк)
- Когда важна скорость выполнения
- Работы с Parquet файлами (оптимизирован для этого)
- Когда нужно обработать данные быстрее
- Параллельных вычислений

**Примеры использования Polars:**
```python
import polars as pl
# Lazy API - ничего не загружается до .collect()
lf = pl.scan_parquet('large_data.parquet')
result = lf.filter(pl.col('revenue') > 1_000_000).collect()
```

### Синтаксические различия:

**Pandas:**
```python
df[df['revenue'] > 1000000]  # Булева индексация
df.groupby('region')['revenue'].mean()  # Группировка
df['new_col'] = df['col1'] + df['col2']  # Создание столбца
```

**Polars:**
```python
df.filter(pl.col('revenue') > 1000000)  # Метод filter
df.group_by('region').agg(pl.col('revenue').mean())  # Группировка
df.with_columns((pl.col('col1') + pl.col('col2')).alias('new_col'))  # Создание столбца
```

### Стратегия для RFSD:

В этом ноутбуке мы используем **гибридный подход**:

1. **Polars для загрузки и обработки больших данных:**
   - Загрузка данных из Parquet файлов
   - Фильтрация и выбор столбцов
   - Группировка и агрегация
   - Расчет финансовых показателей

2. **Pandas для визуализации:**
   - Конвертируем небольшие подвыборки из Polars в pandas
   - Используем matplotlib/seaborn для графиков
   - Pandas удобнее для интерактивной визуализации

**Пример гибридного подхода:**
```python
# Polars для эффективной обработки
df_polars = pl.scan_parquet('data.parquet').filter(...).collect()

# Конвертация в pandas для визуализации
df_pandas = df_polars.to_pandas()
df_pandas.plot()  # Используем pandas для графиков
```

### Миграция между библиотеками:

- **Polars → Pandas:** `.to_pandas()` (просто)
- **Pandas → Polars:** `pl.from_pandas()` (просто)
- Конвертация обычно быстрая для небольших датасетов

### Вывод:

- **Для RFSD (60+ млн строк):** Используем **Polars** — это критически важно для производительности
- **Для визуализации:** Используем **pandas** — удобнее и больше возможностей
- **Для обучения:** Знание обеих библиотек полезно, так как pandas все еще стандарт индустрии

**Рекомендация:** Начните с Polars для больших данных, конвертируйте в pandas только когда нужно для визуализации или специфических функций pandas.


---

## Практическое задание 2: Понимание форматов данных

**Время: 3-5 минут**

**Задание:**
Ответьте на следующие вопросы (устно):

1. Почему для RFSD (60+ млн строк) используется формат Parquet, а не CSV?
2. Что такое "column pruning" и как это помогает при работе с большими данными?
3. В чем основное различие между Pandas и Polars с точки зрения производительности?
4. Что такое LazyFrame в Polars и зачем он нужен?



## Установка необходимых библиотек

**Важно:** RFSD содержит более 60 миллионов строк данных. Для эффективной работы рекомендуется использовать **Polars** вместо pandas, так как он:
- В 5-30 раз быстрее pandas для больших датасетов
- Использует многопоточность по умолчанию
- Эффективно работает с памятью
- Поддерживает lazy evaluation (вычисления откладываются до необходимости)
- Оптимизирован для работы с Parquet файлами


In [20]:
%pip install polars pyarrow datasets


Note: you may need to restart the kernel to use updated packages.


## Импорт библиотек

Используем **Polars** как основной инструмент для работы с данными. Pandas можно использовать для визуализации и финальных преобразований небольших подвыборок.


In [1]:
import polars as pl
import pandas as pd  # Для визуализации и небольших подвыборок
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Настройка Polars для максимальной производительности
pl.Config.set_fmt_str_lengths(100)  # Показывать больше символов в выводе


polars.config.Config

## Загрузка данных RFSD

**Стратегия работы с большими данными:**

1. **Используйте Lazy API Polars** - фильтруйте и выбирайте столбцы ДО загрузки в память
2. **Загружайте только нужные годы** - данные партиционированы по годам
3. **Выбирайте только нужные столбцы** - в RFSD более 200 переменных
4. **Применяйте фильтры на уровне чтения** - это значительно ускоряет работу

Данные находятся локально в папке `RFSD_PATH` и организованы по годам в формате Parquet.


In [2]:
RFSD_PATH = 'F:\\YandexDisk\\Russia\\RFSD'

### Размерность данных RFSD

**В RFSD финансовые данные хранятся в ТЫС. РУБЛЕЙ.**

Это важно учитывать при фильтрации и анализе:
- При фильтрации используйте значения в рублях (например, `1_000_000` для 1 млрд руб.)


In [3]:
def load_rfsd_year_polars(
    year, 
    rfsd_path=RFSD_PATH,
    columns=None,
    filters=None,
    lazy=True
):
    """
    Загружает данные RFSD за указанный год используя Polars
    
    Parameters:
    -----------
    year : int
        Год данных (2011-2024)
    rfsd_path : Path
        Путь к папке с данными RFSD
    columns : list of str, optional
        Список столбцов для загрузки (None = все столбцы)
    filters : list, optional
        Список фильтров Polars для применения при чтении
    lazy : bool
        Если True, возвращает LazyFrame (рекомендуется для больших данных)
        
    Returns:
    --------
    pl.DataFrame or pl.LazyFrame
        DataFrame/LazyFrame с данными за указанный год
    """
    year_path = rfsd_path / f"year={year}"
    
    if not year_path.exists():
        raise ValueError(f"Данные за {year} год не найдены")
    
    # Используем scan_parquet для эффективной загрузки
    # scan_parquet используем .select() после сканирования
    lf = pl.scan_parquet(str(year_path / "*.parquet"))
    
    # Применяем фильтры если указаны (predicate pushdown)
    if filters is not None:
        for filter_expr in filters:
            lf = lf.filter(filter_expr)
    
    # Выбираем только нужные столбцы если указаны
    if columns is not None:
        lf = lf.select(columns)
    
    if lazy:
        print(f"Создан LazyFrame для {year} года")
        if columns:
            print(f"Выбрано {len(columns)} переменных из доступных")
        print("Используйте .collect() для выполнения вычислений")
        return lf
    else:
        df = lf.collect()
        print(f"Загружено {len(df):,} наблюдений за {year} год")
        if columns:
            print(f"Загружено {len(columns)} переменных из доступных")
        else:
            print(f"Загружено {len(df.columns)} переменных")
        return df



In [ ]:
# Пример 1: Загрузка только ключевых столбцов за 2024 год (эффективно!)
key_columns = [
    'inn', 'ogrn', 'region', 'okved', 'okved_section',
    'line_2110',  # Выручка
    'line_2400',  # Чистая прибыль
    'line_1600',  # Активы всего
    'line_1300',  # Капитал и резервы
    'line_1400',  # Долгосрочные обязательства
    'line_1500',  # Краткосрочные обязательства
    'filed', 'eligible'
]

# Сначала создаем LazyFrame (ничего не загружается в память)
lf_2024 = load_rfsd_year_polars(2024, columns=key_columns, lazy=True)

# Теперь можем применять фильтры и только потом загружать
# Например, только компании с поданной отчетностью и выручкой > 1 млн руб.
lf_filtered = lf_2024.filter(
    (pl.col('filed') == 1) & 
    (pl.col('line_2110') > 1)
)

# Только сейчас загружаем данные в память
df_2024_sample = lf_filtered.collect()
print(f"\nЗагружено {len(df_2024_sample):,} компаний после фильтрации")


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
# Пример: просмотр данных для понимания размерности
# Данные RFSD хранятся в рублях

# Загружаем небольшую выборку для просмотра
lf_check = load_rfsd_year_polars(2024, columns=['inn', 'line_2110', 'filed'], lazy=True)
lf_check_filed = lf_check.filter(pl.col('filed') == 1)

# Загружаем первые 10 компаний для просмотра
df_check = lf_check_filed.head(10).collect()

print("Примеры значений выручки (line_2110) - данные в рублях:")
print(df_check.select(['inn', 'line_2110']))

print("\nПримеры значений:")
print("1,563,800,000 рублей = 1.56 млрд руб.")
print("2,699,500,000 рублей = 2.70 млрд руб.")
print("\nДля фильтрации используйте значения в рублях:")
print("  pl.col('line_2110') > 1_000_000_000  # > 1 млрд руб.")


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
# Путь к локальной папке с данными RFSD
RFSD_PATH = Path(r"E:\YandexDisk\Russia\RFSD")

# Проверяем доступные годы
available_years = sorted([int(d.name.split('=')[1]) for d in RFSD_PATH.iterdir() if d.is_dir() and d.name.startswith('year=')])
print(f"Доступные годы: {available_years}")


Доступные годы: [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


---

## Практическое задание 3: Загрузка и фильтрация данных

**Время: 7-10 минут**

**Задание:**
1. Загрузите данные RFSD за 2024 год, используя функцию `load_rfsd_year_polars()`.
2. Создайте фильтр для компаний с выручкой от 500 млн до 2 млрд рублей.
3. Отфильтруйте только компании, которые подали отчетность (`filed == 1`).
4. Загрузите отфильтрованные данные и выведите количество найденных компаний.
5. Выведите первые 5 строк отфильтрованных данных с колонками: `inn`, `region`, `okved_section`, `line_2110`.

**Подсказка:** Используйте LazyFrame API - сначала создайте LazyFrame, затем примените фильтры, и только потом вызовите `.collect()`.


In [ ]:
# Ваше решение здесь:
# 1. Загрузка данных за 2024 год
# lf_2024 = load_rfsd_year_polars(2024, columns=key_columns, lazy=True)

# 2-3. Применение фильтров
# lf_filtered = lf_2024.filter(...)

# 4. Загрузка данных
# df_filtered = lf_filtered.collect()

# 5. Вывод результатов
# print(f"Найдено компаний: {len(df_filtered)}")
# df_filtered.select([...]).head(5)


### Эффективная загрузка данных за конкретный год

**Ключевые переменные RFSD (на основе документации):**

**Идентификаторы:**
- `inn` - ИНН компании (основной идентификатор)
- `ogrn` - ОГРН
- ⚠️ **Названия компаний отсутствуют** - RFSD содержит только идентификаторы

**Классификаторы:**
- `region` - Регион регистрации
- `okved` - ОКВЭД (отрасль)
- `okved_section` - Секция ОКВЭД

**Финансовые показатели:**
- `line_2110` - Выручка (форма №2)
- `line_2400` - Чистая прибыль (убыток) (форма №2)
- `line_1600` - Активы всего (форма №1)
- `line_1300` - Капитал и резервы (форма №1)
- `line_1400` - Долгосрочные обязательства (форма №1)
- `line_1500` - Краткосрочные обязательства (форма №1)

**Метаданные:**
- `filed` - Флаг подачи отчетности (1 = подана, 0 = не подана)
- `eligible` - Флаг обязательности подачи отчетности

Начнем с эффективной загрузки данных за 2024 год (самые свежие данные).


In [ ]:
# функция переименования (фильтрует только существующие столбцы)
def rename_rfsd_columns_fixed(df, use_descriptive_names=True):
    """
    Переименовывает столбцы RFSD в описательные названия (исправленная версия)
    
    Parameters:
    -----------
    df : pl.DataFrame или pl.LazyFrame
        DataFrame с данными RFSD
    use_descriptive_names : bool
        Если True, загружает и применяет описательные названия из GitHub
        
    Returns:
    --------
    pl.DataFrame или pl.LazyFrame
        DataFrame с переименованными столбцами
    """
    if not use_descriptive_names:
        return df
    
    # Загружаем словарь переименований из GitHub репозитория RFSD
    renaming_df = pl.read_csv(
        'https://raw.githubusercontent.com/irlcode/RFSD/main/aux/descriptive_names_dict.csv'
    )
    
    # Создаем полный словарь для переименования
    full_rename_dict = {
        row[0]: row[1] 
        for row in renaming_df.select(['original', 'descriptive']).iter_rows()
    }
    
    # Получаем список столбцов DataFrame (работает и для DataFrame, и для LazyFrame)
    if hasattr(df, 'columns'):
        # DataFrame
        available_columns = df.columns
    elif hasattr(df, 'schema'):
        # LazyFrame - получаем столбцы из схемы
        available_columns = list(df.schema.keys())
    else:
        available_columns = []
    
    # Фильтруем словарь: оставляем только те столбцы, которые есть в DataFrame
    rename_dict = {
        old_name: new_name 
        for old_name, new_name in full_rename_dict.items() 
        if old_name in available_columns
    }
    
    if not rename_dict:
        print("Предупреждение: не найдено столбцов для переименования")
        return df
    
    # Применяем переименование с strict=False для безопасности
    df_renamed = df.rename(rename_dict, strict=False)
    
    print(f"Переименовано {len(rename_dict)} из {len(full_rename_dict)} доступных столбцов")
    print("Примеры переименований:")
    sample_renames = list(rename_dict.items())[:5]
    for orig, desc in sample_renames:
        print(f"  {orig} → {desc}")
    
    return df_renamed

# Обновляем оригинальную функцию (перезаписываем)
rename_rfsd_columns = rename_rfsd_columns_fixed


### Пример использования переименования столбцов

Покажем, как работает функция переименования на практике.


In [ ]:
# Пример: Переименование столбцов в описательные названия

# Сначала посмотрим на текущие названия столбцов
print("Столбцы ДО переименования:")
print(df_2024_sample.columns)
print(f"\nПервые 3 строки с техническими названиями:")
print(df_2024_sample.select(['inn', 'region', 'line_2110', 'line_2400', 'line_1600']).head(3))

# Применяем переименование
df_2024_renamed = rename_rfsd_columns(df_2024_sample, use_descriptive_names=True)

# Посмотрим на результат
print("\n" + "="*80)
print("Столбцы ПОСЛЕ переименования:")
print(df_2024_renamed.columns)

# Пример работы с переименованными данными
print("\n" + "="*80)
print("Работа с переименованными данными:")
print("\nПервые 3 строки с описательными названиями:")
# Обратите внимание: теперь используем описательные названия вместо line_2110, line_2400 и т.д.
# Но нужно знать, как они переименовались - проверим это
print(df_2024_renamed.head(3))


Столбцы ДО переименования:
['inn', 'ogrn', 'region', 'okved', 'okved_section', 'line_2110', 'line_2400', 'line_1600', 'line_1300', 'line_1400', 'line_1500', 'filed', 'eligible']

Первые 3 строки с техническими названиями:
shape: (3, 5)
┌────────────┬──────────────────┬───────────┬──────────────┬───────────┐
│ inn        ┆ region           ┆ line_2110 ┆ line_2400    ┆ line_1600 │
│ ---        ┆ ---              ┆ ---       ┆ ---          ┆ ---       │
│ str        ┆ str              ┆ f64       ┆ f64          ┆ f64       │
╞════════════╪══════════════════╪═══════════╪══════════════╪═══════════╡
│ 1644003838 ┆ tatarstan        ┆ 1.5638e9  ┆ 2.51375287e8 ┆ 1.4931e9  │
│ 2310031475 ┆ krasnodar        ┆ 2.6995e9  ┆ 4.6419038e7  ┆ 1.3487e9  │
│ 5003021311 ┆ sankt-petersburg ┆ 1.4204e9  ┆ -3.8949886e7 ┆ 1.7191e9  │
└────────────┴──────────────────┴───────────┴──────────────┴───────────┘
Переименовано 12 из 211 доступных столбцов
Примеры переименований:
  inn → inn
  ogrn → ogrn
  region → reg

In [ ]:
# Практический пример: использование переименованных данных для анализа

# После переименования можно работать с более понятными названиями столбцов
# Но нужно знать, как они переименовались - обычно это описательные названия на русском

# Пример: если line_2110 переименовалась в "Выручка", можно использовать:
# df_2024_renamed.select(['inn', 'region', 'Выручка', 'Чистая прибыль'])

# Для проверки, какие столбцы были переименованы, можно сравнить:
print("Сравнение столбцов:")
print(f"\nИсходные столбцы (первые 10): {df_2024_sample.columns[:10]}")
print(f"\nПереименованные столбцы (первые 10): {df_2024_renamed.columns[:10]}")

# Важно: если вы хотите использовать переименованные данные дальше,
# сохраните результат в переменную или перезапишите исходную:
# df_2024_sample = rename_rfsd_columns(df_2024_sample, use_descriptive_names=True)

# Или работайте с отдельной копией:
# df_renamed = rename_rfsd_columns(df_2024_sample.copy(), use_descriptive_names=True)


Сравнение столбцов:

Исходные столбцы (первые 10): ['inn', 'ogrn', 'region', 'okved', 'okved_section', 'line_2110', 'line_2400', 'line_1600', 'line_1300', 'line_1400']

Переименованные столбцы (первые 10): ['inn', 'ogrn', 'region', 'okved', 'okved_section', 'PL_revenue', 'PL_net_profit', 'B_assets', 'B_total_equity', 'B_longterm_liab']


### Получение названий компаний

**Важно:** RFSD содержит только идентификаторы компаний (ИНН, ОГРН), но не содержит названий компаний напрямую.

**Для получения названий компаний можно:**

1. **Использовать ЕГРЮЛ (Единый государственный реестр юридических лиц):**
   - По ИНН или ОГРН можно получить полную информацию о компании
   - Доступ через API ФНС или коммерческие сервисы (СПАРК, Контур.Фокус и др.)

2. **Использовать дополнительные источники данных:**
   - Коммерческие базы данных (СПАРК, Интерфакс)
   - Поиск в интернете

3. **Для публичных компаний:**
   - Можно использовать тикеры и названия из данных биржи
   - Сопоставить по ИНН/ОГРН

**В этом ноутбуке** мы будем работать с идентификаторами (ИНН, ОГРН), которые являются уникальными идентификаторами компаний в России.


In [ ]:
df_2024_sample

inn,ogrn,region,okved,okved_section,line_2110,line_2400,line_1600,line_1300,line_1400,line_1500,filed,eligible
str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""1644003838""","""1021601623702""","""tatarstan""","""06.10""","""B""",1.5638e9,2.51375287e8,1.4931e9,9.92130682e8,9.7463751e7,4.03523242e8,1.0,1.0
"""2310031475""","""1022301598549""","""krasnodar""","""47.11""","""G""",2.6995e9,4.6419038e7,1.3487e9,1.61487762e8,4.6123619e8,7.26019103e8,1.0,1.0
"""5003021311""","""1025000653930""","""sankt-petersburg""","""46.71""","""G""",1.4204e9,-3.8949886e7,1.7191e9,9.09256245e8,2.98050051e8,5.11775935e8,1.0,1.0
"""5003052454""","""1045000923967""","""moscow reg.""","""46.35""","""G""",1.2493e9,3.6567384e7,1.75800073e8,3.205184e7,3.083758e6,1.40664475e8,1.0,1.0
"""5902201970""","""1035900103997""","""perm""","""06.10.10""","""B""",1.4438e9,1.40224536e8,1.1569e9,8.87126038e8,9.800005e7,1.71744096e8,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…
"""7708004767""","""1027700035769""","""moscow city""","""70.10.20""","""M""",3.0469e9,7.32516214e8,2.8639e9,1.3399e9,1.33935159e8,1.3901e9,1.0,1.0
"""7727547261""","""1057747421247""","""tyumen""","""19.20""","""C""",1.2122e9,1.66616624e8,2.0963e9,8.96036038e8,6.45294116e8,5.54974535e8,1.0,1.0
"""7736050003""","""1027700070518""","""sankt-petersburg""","""46.71""","""G""",6.2566e9,-1.0763e9,2.6163e10,1.6330e10,6.6697e9,3.1628e9,1.0,1.0


In [ ]:
# Посмотрим на структуру данных
df_2024_sample.head()


inn,ogrn,region,okved,okved_section,line_2110,line_2400,line_1600,line_1300,line_1400,line_1500,filed,eligible
str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""1644003838""","""1021601623702""","""tatarstan""","""06.10""","""B""",1.5638e9,2.51375287e8,1.4931e9,9.92130682e8,9.7463751e7,4.03523242e8,1.0,1.0
"""2310031475""","""1022301598549""","""krasnodar""","""47.11""","""G""",2.6995e9,4.6419038e7,1.3487e9,1.61487762e8,4.6123619e8,7.26019103e8,1.0,1.0
"""5003021311""","""1025000653930""","""sankt-petersburg""","""46.71""","""G""",1.4204e9,-3.8949886e7,1.7191e9,9.09256245e8,2.98050051e8,5.11775935e8,1.0,1.0
"""5003052454""","""1045000923967""","""moscow reg.""","""46.35""","""G""",1.2493e9,3.6567384e7,1.75800073e8,3.205184e7,3.083758e6,1.40664475e8,1.0,1.0
"""5902201970""","""1035900103997""","""perm""","""06.10.10""","""B""",1.4438e9,1.40224536e8,1.1569e9,8.87126038e8,9.800005e7,1.71744096e8,1.0,1.0


In [ ]:
# Информация о данных (Polars)
print(f"Размер: {df_2024_sample.shape[0]:,} строк × {df_2024_sample.shape[1]} столбцов")
print(f"\nСтолбцы:")
print(df_2024_sample.columns)
print(f"\nТипы данных:")
print(df_2024_sample.schema)


Размер: 11 строк × 13 столбцов

Столбцы:
['inn', 'ogrn', 'region', 'okved', 'okved_section', 'line_2110', 'line_2400', 'line_1600', 'line_1300', 'line_1400', 'line_1500', 'filed', 'eligible']

Типы данных:
Schema({'inn': String, 'ogrn': String, 'region': String, 'okved': String, 'okved_section': String, 'line_2110': Float64, 'line_2400': Float64, 'line_1600': Float64, 'line_1300': Float64, 'line_1400': Float64, 'line_1500': Float64, 'filed': Float64, 'eligible': Float64})


In [ ]:
# Размер таблицы
print(f"Размер таблицы: {df_2024_sample.shape[0]:,} строк × {df_2024_sample.shape[1]} столбцов")
print(f"Использовано памяти: {df_2024_sample.estimated_size('mb'):.2f} MB")

# Базовые статистики (Polars очень быстрый для этого!)
print("\nОписательные статистики:")
df_2024_sample.describe()


Размер таблицы: 11 строк × 13 столбцов
Использовано памяти: 0.00 MB

Описательные статистики:


statistic,inn,ogrn,region,okved,okved_section,line_2110,line_2400,line_1600,line_1300,line_1400,line_1500,filed,eligible
str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""11""","""11""","""11""","""11""","""11""",11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,2.2359e9,3.7377e7,3.5931e9,2.0539e9,8.4165e8,6.9763e8,1.0,1.0
"""std""",null,null,null,null,null,1.5073e9,4.2530e8,7.5272e9,4.7585e9,1.9490e9,9.0068e8,0.0,0.0
"""min""","""1644003838""","""1021601623702""","""khanty-mansijsk""","""06.10""","""B""",1.2122e9,-1.0763e9,1.228132e6,1.112185e6,0.0,115946.0,1.0,1.0
"""25%""",null,null,null,null,null,1.4204e9,3.6567384e7,1.2422e9,1.61487762e8,9.800005e7,1.76115867e8,1.0,1.0
"""50%""",null,null,null,null,null,1.4916e9,4.6419038e7,1.3487e9,8.96036038e8,1.33935159e8,4.36159585e8,1.0,1.0
"""75%""",null,null,null,null,null,2.9199e9,1.66616624e8,2.0963e9,9.92130682e8,6.45294116e8,7.26019103e8,1.0,1.0
"""max""","""8608048498""","""1057748136027""","""tyumen""","""70.10.20""","""M""",6.2566e9,7.32516214e8,2.6163e10,1.6330e10,6.6697e9,3.1628e9,1.0,1.0


### Просмотр доступных переменных

RFSD содержит стандартизированные переменные из форм бухгалтерской отчетности:
- Форма №1 (Бухгалтерский баланс)
- Форма №2 (Отчет о финансовых результатах)
- Форма №4 (Отчет о движении денежных средств)


In [ ]:
# Для просмотра всех доступных столбцов можно загрузить схему без данных
# Это очень быстро!
year_path = RFSD_PATH / "year=2024"
sample_file = list(year_path.glob("*.parquet"))[0]
schema = pl.read_parquet(sample_file, n_rows=0).schema

print(f"Всего переменных в RFSD: {len(schema)}")
print("\nКлючевые переменные для финансового анализа:")

# Поиск переменных с названиями компаний
name_keywords = ['name', 'title', 'company', 'firm', 'organization', 'org', 'название', 'наименование']
name_cols = [k for k in schema.keys() if any(keyword in k.lower() for keyword in name_keywords)]
if name_cols:
    print("\n📌 Названия компаний:")
    for col in name_cols:
        print(f"  - {col}")
else:
    print("\n⚠️  Переменная с названием компании не найдена в схеме")
    print("   RFSD использует только идентификаторы (ИНН, ОГРН)")
    print("   Для получения названий можно использовать ЕГРЮЛ по ИНН/ОГРН")

print("\nИдентификаторы:")
id_cols = [k for k in schema.keys() if k in ['inn', 'ogrn', 'okpo', 'okopf', 'okfc']]
for col in id_cols:
    print(f"  - {col}")

print("\nКлассификаторы:")
class_cols = [k for k in schema.keys() if k in ['region', 'region_taxcode', 'okved', 'okved_section']]
for col in class_cols:
    print(f"  - {col}")

print("\nФинансовые показатели (форма №1 - баланс):")
balance_cols = [k for k in schema.keys() if k.startswith('line_1')]
for col in sorted(balance_cols)[:10]:
    print(f"  - {col}")
print("  ...")

print("\nФинансовые показатели (форма №2 - прибыли и убытки):")
pl_cols = [k for k in schema.keys() if k.startswith('line_2')]
for col in sorted(pl_cols)[:10]:
    print(f"  - {col}")
print("  ...")

# Показываем все нестандартные столбцы (не line_*)
print("\nДругие переменные:")
other_cols = [k for k in schema.keys() if not k.startswith('line_') and k not in id_cols + class_cols + name_cols]
for col in sorted(other_cols)[:20]:  # Показываем первые 20
    print(f"  - {col}")
if len(other_cols) > 20:
    print(f"  ... и еще {len(other_cols) - 20} переменных")


Всего переменных в RFSD: 213

Ключевые переменные для финансового анализа:

⚠️  Переменная с названием компании не найдена в схеме
   RFSD использует только идентификаторы (ИНН, ОГРН)
   Для получения названий можно использовать ЕГРЮЛ по ИНН/ОГРН

Идентификаторы:
  - inn
  - ogrn
  - okpo
  - okopf
  - okfc

Классификаторы:
  - region
  - region_taxcode
  - okved
  - okved_section

Финансовые показатели (форма №1 - баланс):
  - line_1100
  - line_1110
  - line_1120
  - line_1130
  - line_1140
  - line_1150
  - line_1160
  - line_1170
  - line_1180
  - line_1190
  ...

Финансовые показатели (форма №2 - прибыли и убытки):
  - line_2100
  - line_2110
  - line_2120
  - line_2200
  - line_2210
  - line_2220
  - line_2300
  - line_2310
  - line_2320
  - line_2330
  ...

Другие переменные:
  - age
  - articulated
  - creation_date
  - dissolution_date
  - eligible
  - exemption_criteria
  - filed
  - financial
  - geocoding_quality
  - imputed
  - lat
  - lon
  - okogu
  - oktmo
  - outlier
 

## Работа с данными

### Выбор переменных и сортировка строк

Выберем ключевые переменные для анализа финансового состояния компаний.


In [ ]:
# Структура переменных RFSD основана на строках форм отчетности:
# - line_1xxx - Форма №1 (Бухгалтерский баланс)
# - line_2xxx - Форма №2 (Отчет о финансовых результатах)  
# - line_3xxx - Форма №4 (Отчет о движении денежных средств)

# Ключевые переменные для анализа:
RFSD_VARIABLES = {
    # Идентификаторы
    'inn': 'ИНН компании',
    'ogrn': 'ОГРН',
    'region': 'Регион регистрации',
    'okved': 'ОКВЭД',
    'okved_section': 'Секция ОКВЭД',
    
    # Форма №2 (Прибыли и убытки)
    'line_2110': 'Выручка',
    'line_2120': 'Себестоимость продаж',
    'line_2200': 'Прибыль (убыток) от продаж',
    'line_2300': 'Прибыль (убыток) до налогообложения',
    'line_2400': 'Чистая прибыль (убыток)',
    
    # Форма №1 (Баланс) - Активы
    'line_1100': 'Внеоборотные активы',
    'line_1200': 'Оборотные активы',
    'line_1600': 'Активы всего',
    
    # Форма №1 (Баланс) - Пассивы
    'line_1300': 'Капитал и резервы',
    'line_1400': 'Долгосрочные обязательства',
    'line_1500': 'Краткосрочные обязательства',
    'line_1700': 'Пассивы всего',
}

print("Ключевые переменные RFSD для финансового анализа:")
for var, desc in RFSD_VARIABLES.items():
    print(f"  {var:20s} - {desc}")


**Примечание:** Структура переменных в RFSD может отличаться от СПАРК. 
Рекомендуется изучить документацию RFSD или использовать интерактивный просмотр переменных.

Давайте посмотрим на основные идентификаторы и финансовые показатели.


In [ ]:
# Пример работы с данными в Polars
# Polars использует другой синтаксис, но он более эффективный

# Выборка данных
df_2024_sample.select([
    'inn', 'region', 'okved_section', 
    'line_2110', 'line_2400', 'line_1600'
]).head(10)


inn,region,okved_section,line_2110,line_2400,line_1600
str,str,str,f64,f64,f64
"""1644003838""","""tatarstan""","""B""",1.5638e9,2.51375287e8,1.4931e9
"""2310031475""","""krasnodar""","""G""",2.6995e9,4.6419038e7,1.3487e9
"""5003021311""","""sankt-petersburg""","""G""",1.4204e9,-3.8949886e7,1.7191e9
"""5003052454""","""moscow reg.""","""G""",1.2493e9,3.6567384e7,1.75800073e8
"""5902201970""","""perm""","""B""",1.4438e9,1.40224536e8,1.1569e9
"""7703562479""","""moscow city""","""K""",1.4916e9,1.111994e6,1.228132e6
"""7708004767""","""moscow city""","""M""",3.0469e9,7.32516214e8,2.8639e9
"""7727547261""","""tyumen""","""C""",1.2122e9,1.66616624e8,2.0963e9
"""7736050003""","""sankt-petersburg""","""G""",6.2566e9,-1.0763e9,2.6163e10


### Загрузка данных за несколько лет

Для анализа динамики можно загрузить данные за несколько лет.


---

## 📝 Практическое задание 4: Анализ распределения данных

**Время: 5-7 минут**

**Задание:**
Используя загруженные данные `df_2024_sample`:

1. Найдите топ-5 регионов по количеству компаний в выборке.
2. Найдите топ-5 отраслей (секций ОКВЭД) по количеству компаний.
3. Посчитайте среднюю выручку (`line_2110`) по регионам и отсортируйте по убыванию.
4. Посчитайте среднюю выручку по отраслям (секциям ОКВЭД).

**Подсказка:** Используйте методы `.group_by()`, `.agg()`, `.sort()` в Polars.



In [ ]:
# Ваше решение здесь:
# 1. Топ-5 регионов
# ...

# 2. Топ-5 отраслей
# ...

# 3. Средняя выручка по регионам
# ...

# 4. Средняя выручка по отраслям
# ...


---

## Практическое задание 5: Расчет финансовых показателей

**Время: 7-10 минут**

**Задание:**
Используя функцию `calculate_financial_ratios_polars()` или написав свой код:

1. Рассчитайте следующие показатели для компаний из `df_2024_sample`:
   - Рентабельность продаж (ROS) = `line_2400` / `line_2110`
   - Рентабельность активов (ROA) = `line_2400` / `line_1600`
   - Коэффициент текущей ликвидности = `line_1200` / `line_1500` (если есть `line_1200`)

2. Отфильтруйте только компании с валидными значениями (исключите бесконечные значения и деление на ноль).

3. Найдите компанию с максимальной ROS и компанию с максимальной ROA.

4. Посчитайте средние значения ROS и ROA по отраслям (секциям ОКВЭД).

**Подсказка:** Используйте `.with_columns()` для добавления новых столбцов, `.filter()` для фильтрации, `.is_finite()` для проверки валидности.

**Проверка:** Покажите результаты преподавателю.


In [ ]:
# Ваше решение здесь:
# 1. Расчет показателей
# df_with_ratios = df_2024_sample.with_columns([
#     (pl.col('line_2400') / pl.col('line_2110')).alias('ros'),
#     ...
# ])

# 2. Фильтрация валидных значений
# df_valid = df_with_ratios.filter(...)

# 3. Поиск максимумов
# ...

# 4. Средние по отраслям
# ...


---

## 📝 Практическое задание 6: Визуализация данных

**Время: 5-7 минут**

**Задание:**
Используя данные с рассчитанными финансовыми показателями:

1. Постройте гистограмму распределения выручки (`line_2110`). Используйте логарифмическую шкалу по оси Y.

2. Постройте scatter plot зависимости ROS от ROA (ROS по оси X, ROA по оси Y).

3. Постройте столбчатую диаграмму средних значений ROS по отраслям (секциям ОКВЭД).

**Подсказка:** 
- Конвертируйте данные в pandas для визуализации: `.to_pandas()`
- Используйте `plt.hist()`, `plt.scatter()`, `plt.bar()` из matplotlib
- Не забудьте добавить подписи осей и заголовки

**Проверка:** Покажите графики преподавателю.


In [ ]:
# Ваше решение здесь:
# import matplotlib.pyplot as plt

# 1. Гистограмма выручки
# ...

# 2. Scatter plot ROS vs ROA
# ...

# 3. Столбчатая диаграмма по отраслям
# ...


## Анализ данных

### Базовые статистики

После загрузки данных важно понять их структуру и провести первичный анализ.


In [ ]:
# Проверка на пропущенные значения (Polars)
missing_stats = (
    df_2024_sample
    .null_count()
    .transpose(include_header=True, column_names=['Пропущено'])
    .with_columns([
        (pl.col('Пропущено') / len(df_2024_sample) * 100).alias('Процент')
    ])
    .filter(pl.col('Пропущено') > 0)
    .sort('Пропущено', descending=True)
)

missing_stats.head(20)


### Уникальные значения

Посмотрим на распределение компаний по регионам и отраслям.


In [ ]:
# Анализ распределения по регионам (Polars очень быстрый!)
print("Топ-10 регионов по количеству компаний:")
(
    df_2024_sample
    .group_by('region')
    .agg(pl.count().alias('количество'))
    .sort('количество', descending=True)
    .head(10)
)

# Анализ по отраслям (секциям ОКВЭД)
print("\nТоп-10 отраслей по количеству компаний:")
(
    df_2024_sample
    .group_by('okved_section')
    .agg(pl.count().alias('количество'))
    .sort('количество', descending=True)
    .head(10)
)


## Расчет финансовых показателей

После изучения структуры данных можно рассчитать финансовые мультипликаторы и показатели.

### Примеры финансовых показателей:

1. **Рентабельность:**
   - Рентабельность продаж (ROS) = Прибыль / Выручка
   - Рентабельность активов (ROA) = Прибыль / Активы

2. **Ликвидность:**
   - Текущая ликвидность = Оборотные активы / Краткосрочные обязательства
   - Быстрая ликвидность = (Оборотные активы - Запасы) / Краткосрочные обязательства

3. **Долговая нагрузка:**
   - Долг/Активы = Обязательства / Активы
   - Долг/Собственный капитал = Обязательства / Капитал

**Важно:** Точные названия переменных нужно проверить в документации RFSD или путем изучения данных.


In [ ]:
# Расчет финансовых показателей в Polars (очень эффективно!)
# Polars позволяет делать это на уровне LazyFrame до загрузки данных

def calculate_financial_ratios_polars(lf):
    """
    Рассчитывает финансовые показатели используя Polars LazyFrame
    
    Parameters:
    -----------
    lf : pl.LazyFrame
        LazyFrame с финансовыми данными
        
    Returns:
    --------
    pl.LazyFrame
        LazyFrame с добавленными финансовыми показателями
    """
    return lf.with_columns([
        # Рентабельность продаж (ROS)
        (pl.col('line_2400') / pl.col('line_2110')).alias('ros'),
        
        # Рентабельность активов (ROA)
        (pl.col('line_2400') / pl.col('line_1600')).alias('roa'),
        
        # Долговая нагрузка (Leverage)
        ((pl.col('line_1400') + pl.col('line_1500')) / pl.col('line_1600')).alias('leverage'),
        
        # Долг/Собственный капитал
        ((pl.col('line_1400') + pl.col('line_1500')) / pl.col('line_1300')).alias('debt_to_equity'),
        
        # Текущая ликвидность (Current Ratio)
        (pl.col('line_1200') / pl.col('line_1500')).alias('current_ratio'),
    ])

# Применяем расчеты на уровне LazyFrame
lf_with_ratios = calculate_financial_ratios_polars(lf_filtered)

# Фильтруем только валидные значения (избегаем деления на ноль)
lf_valid_ratios = lf_with_ratios.filter(
    (pl.col('ros').is_finite()) &
    (pl.col('roa').is_finite()) &
    (pl.col('leverage').is_finite())
)

# Загружаем только результат
df_with_ratios = lf_valid_ratios.collect()
print(f"Рассчитаны показатели для {len(df_with_ratios):,} компаний")

# Показываем результаты
df_with_ratios.select([
    'inn', 'region', 'okved_section',
    'line_2110', 'line_2400', 'line_1600',
    'ros', 'roa', 'leverage', 'debt_to_equity'
]).head(10)


## Визуализация данных

### Примеры графиков для анализа финансовых данных


---

## 📝 Практическое задание 7: Применение изученного

**Время: 7-10 минут**

**Задание:**
Выполните один из следующих анализов (на выбор):

**Вариант А:** Найдите топ-5 компаний по выручке в вашей выборке и сравните их финансовые показатели (ROS, ROA, леверидж).

**Вариант Б:** Сравните средние финансовые показатели двух разных отраслей (секций ОКВЭД) из вашей выборки.

**Вариант В:** Найдите все компании с отрицательной прибылью (`line_2400 < 0`) и проанализируйте их распределение по отраслям.

**Вариант Г:** Рассчитайте среднюю выручку и среднюю прибыль по регионам и определите топ-3 региона по этим показателям.

**Проверка:** Покажите результаты и кратко объясните свои выводы преподавателю или соседу.


In [ ]:
# Ваше решение здесь:
# Выберите один из вариантов и выполните анализ

# Вариант А: Топ-5 компаний
# ...

# Вариант Б: Сравнение отраслей
# ...

# Вариант В: Компании с убытками
# ...

# Вариант Г: Анализ по регионам
# ...


In [ ]:
# Визуализация данных
# Для визуализации можно конвертировать небольшие подвыборки в pandas

# Пример: гистограмма распределения выручки
plt.figure(figsize=(12, 6))
# Конвертируем в pandas для визуализации (только для небольших подвыборок!)
df_pd = df_2024_sample.select(['line_2110']).to_pandas()
df_pd['line_2110'].hist(bins=50, edgecolor='black')
plt.xlabel('Выручка, млрд руб.')
plt.ylabel('Количество компаний')
plt.title('Распределение выручки компаний в 2024 году (выручка > 1 млрд руб.)')
plt.yscale('log')
plt.show()

# Пример: scatter plot рентабельности
if 'df_with_ratios' in locals():
    plt.figure(figsize=(10, 6))
    df_ratios_pd = df_with_ratios.select(['ros', 'roa']).to_pandas()
    plt.scatter(df_ratios_pd['ros'], df_ratios_pd['roa'], alpha=0.5)
    plt.xlabel('Рентабельность продаж (ROS)')
    plt.ylabel('Рентабельность активов (ROA)')
    plt.title('Связь между ROS и ROA')
    plt.grid(True, alpha=0.3)
    plt.show()


---

## 📚 План следующего занятия

### Темы для изучения:

1. **Временной анализ (Time Series Analysis)**
   - Анализ динамики финансовых показателей за несколько лет
   - Построение трендов и выявление закономерностей
   - Сравнение показателей до и после кризисных периодов

2. **Региональный анализ**
   - Сравнение финансовых показателей по регионам России
   - Выявление регионов-лидеров и регионов-аутсайдеров
   - Анализ концентрации бизнеса по регионам

3. **Отраслевой анализ (расширенный)**
   - Глубокий анализ конкретных отраслей (нефтегаз, торговля, IT и т.д.)
   - Сравнение отраслевых показателей с общероссийскими
   - Выявление отраслевых особенностей и закономерностей

4. **Работа с выбросами и аномалиями**
   - Поиск компаний с экстремальными показателями
   - Обработка выбросов в данных
   - Анализ причин аномальных значений

5. **Продвинутая визуализация**
   - Интерактивные графики (plotly)
   - Географическая визуализация (карты регионов)
   - Комплексные дашборды

6. **Экспорт и сохранение результатов**
   - Сохранение обработанных данных в различных форматах
   - Создание отчетов
   - Работа с Excel

### Практические задания для следующего занятия:

**Задание 1: Анализ динамики отрасли**
- Выберите одну отрасль (секцию ОКВЭД)
- Проанализируйте динамику средних показателей за 2020-2024 годы
- Постройте графики трендов
- Объясните изменения

**Задание 2: Региональный рейтинг**
- Создайте рейтинг регионов по финансовым показателям
- Определите топ-5 и аутсайдеров
- Визуализируйте результаты на карте или графиках

**Задание 3: Поиск инвестиционных возможностей**
- Найдите компании с высоким потенциалом роста
- Критерии: растущая выручка, положительная динамика прибыли, разумная долговая нагрузка
- Создайте список из 10-20 компаний с обоснованием

**Задание 4: Отраслевой бенчмарк**
- Выберите 3-5 компаний из одной отрасли
- Сравните их показатели между собой и со средними по отрасли
- Определите лидера и объясните причины успеха

**Задание 5: Анализ кризисных периодов**
- Сравните показатели компаний в 2020-2021 (пандемия) и 2022-2023 (санкции)
- Определите, какие отрасли пострадали больше всего
- Какие отрасли показали устойчивость?

---

**Подготовка к следующему занятию:**
- Повторите материал по работе с Polars LazyFrame
- Изучите документацию по matplotlib/seaborn для визуализации
- Подумайте, какие отрасли или регионы вас интересуют для анализа


## Практические примеры использования RFSD

В этом разделе представлены типичные задачи анализа финансовых данных компаний с использованием RFSD. Примеры основаны на реальных use cases из репозитория [RFSD](https://github.com/irlcode/RFSD/tree/main/use_cases).

### Пример 1: Анализ по отраслям (ОКВЭД)

Сравнение средних финансовых показателей между отраслями экономики.


In [ ]:
# Пример 1: Сравнение отраслей по финансовым показателям
# Используем данные с рассчитанными показателями

if 'df_with_ratios' in locals():
    # Группировка по секциям ОКВЭД и расчет средних показателей
    industry_analysis = (
        df_with_ratios
        .group_by('okved_section')
        .agg([
            pl.count().alias('количество_компаний'),
            pl.mean('line_2110').alias('средняя_выручка'),
            pl.mean('line_2400').alias('средняя_прибыль'),
            pl.mean('ros').alias('средняя_ROS'),
            pl.mean('roa').alias('средняя_ROA'),
            pl.mean('leverage').alias('средний_леверидж'),
        ])
        .filter(pl.col('количество_компаний') >= 3)  # Минимум 3 компании в отрасли
        .sort('средняя_ROA', descending=True)
    )
    
    print("Сравнение отраслей по финансовым показателям:")
    print(industry_analysis)
    
    # Визуализация: сравнение ROS по отраслям
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 6))
    industry_pd = industry_analysis.select(['okved_section', 'средняя_ROS']).to_pandas()
    plt.bar(industry_pd['okved_section'], industry_pd['средняя_ROS'])
    plt.xlabel('Секция ОКВЭД')
    plt.ylabel('Средняя рентабельность продаж (ROS)')
    plt.title('Сравнение рентабельности продаж по отраслям')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Сначала выполните расчет финансовых показателей (ячейка выше)")


### Пример 2: Поиск компаний по критериям

Найти компании, соответствующие определенным финансовым критериям (например, высокая рентабельность и низкая долговая нагрузка).


In [ ]:
# Пример 2: Поиск "здоровых" компаний
# Критерии: высокая рентабельность, низкая долговая нагрузка, хорошая ликвидность

if 'df_with_ratios' in locals():
    # Определяем критерии поиска
    healthy_companies = (
        df_with_ratios
        .filter(
            (pl.col('ros') > 0.1) &           # ROS > 10%
            (pl.col('roa') > 0.05) &          # ROA > 5%
            (pl.col('leverage') < 0.7) &      # Леверидж < 70%
            (pl.col('current_ratio') > 1.2) & # Текущая ликвидность > 1.2
            (pl.col('line_2110') > 100_000_000)  # Выручка > 100 млн руб.
        )
        .select([
            'inn', 'region', 'okved_section',
            'line_2110', 'line_2400',
            'ros', 'roa', 'leverage', 'current_ratio'
        ])
        .sort('ros', descending=True)
        .head(20)
    )
    
    print(f"Найдено {len(healthy_companies)} компаний с высокими показателями:")
    print(healthy_companies)
    
    # Анализ распределения по отраслям
    print("\nРаспределение найденных компаний по отраслям:")
    (
        healthy_companies
        .group_by('okved_section')
        .agg(pl.count().alias('количество'))
        .sort('количество', descending=True)
    )
else:
    print("Сначала выполните расчет финансовых показателей")


### Пример 3: Топ компании по показателям

Найти лидеров по различным финансовым показателям (выручка, прибыль, рентабельность).


In [ ]:
# Пример 3: Топ-10 компаний по различным показателям

if 'df_with_ratios' in locals():
    print("=" * 80)
    print("ТОП-10 КОМПАНИЙ ПО РАЗЛИЧНЫМ ПОКАЗАТЕЛЯМ")
    print("=" * 80)
    
    # Топ по выручке
    print("\n1. Топ-10 по выручке:")
    top_revenue = (
        df_with_ratios
        .select(['inn', 'region', 'okved_section', 'line_2110', 'ros', 'roa'])
        .sort('line_2110', descending=True)
        .head(10)
        .with_columns([
            (pl.col('line_2110') / 1_000_000_000).alias('выручка_млрд')
        ])
    )
    print(top_revenue.select(['inn', 'okved_section', 'выручка_млрд', 'ros', 'roa']))
    
    # Топ по прибыли
    print("\n2. Топ-10 по прибыли:")
    top_profit = (
        df_with_ratios
        .filter(pl.col('line_2400') > 0)
        .select(['inn', 'region', 'okved_section', 'line_2400', 'ros', 'roa'])
        .sort('line_2400', descending=True)
        .head(10)
        .with_columns([
            (pl.col('line_2400') / 1_000_000_000).alias('прибыль_млрд')
        ])
    )
    print(top_profit.select(['inn', 'okved_section', 'прибыль_млрд', 'ros', 'roa']))
    
    # Топ по рентабельности продаж (ROS)
    print("\n3. Топ-10 по рентабельности продаж (ROS):")
    top_ros = (
        df_with_ratios
        .filter(
            (pl.col('ros').is_finite()) &
            (pl.col('ros') > 0) &
            (pl.col('line_2110') > 100_000_000)  # Минимум 100 млн выручки
        )
        .select(['inn', 'region', 'okved_section', 'line_2110', 'ros', 'roa'])
        .sort('ros', descending=True)
        .head(10)
        .with_columns([
            (pl.col('line_2110') / 1_000_000_000).alias('выручка_млрд'),
            (pl.col('ros') * 100).alias('ROS_процент')
        ])
    )
    print(top_ros.select(['inn', 'okved_section', 'выручка_млрд', 'ROS_процент', 'roa']))
    
    # Топ по рентабельности активов (ROA)
    print("\n4. Топ-10 по рентабельности активов (ROA):")
    top_roa = (
        df_with_ratios
        .filter(
            (pl.col('roa').is_finite()) &
            (pl.col('roa') > 0) &
            (pl.col('line_1600') > 100_000_000)  # Минимум 100 млн активов
        )
        .select(['inn', 'region', 'okved_section', 'line_1600', 'ros', 'roa'])
        .sort('roa', descending=True)
        .head(10)
        .with_columns([
            (pl.col('line_1600') / 1_000_000_000).alias('активы_млрд'),
            (pl.col('roa') * 100).alias('ROA_процент')
        ])
    )
    print(top_roa.select(['inn', 'okved_section', 'активы_млрд', 'ros', 'ROA_процент']))
else:
    print("Сначала выполните расчет финансовых показателей")


### Пример 4: Сравнительный анализ компаний одной отрасли

Сравнение финансовых показателей компаний из одной отрасли для выявления лидеров и аутсайдеров.


In [ ]:
# Пример 4: Сравнение компаний одной отрасли
# Выберем секцию G (Торговля оптовая и розничная) как пример

if 'df_with_ratios' in locals():
    # Фильтруем компании из секции G
    sector_g = (
        df_with_ratios
        .filter(pl.col('okved_section') == 'G')
        .filter(pl.col('line_2110') > 500_000_000)  # Выручка > 500 млн
    )
    
    print(f"Найдено {len(sector_g)} компаний в секции G (Торговля)")
    
    if len(sector_g) > 0:
        # Сравнительная таблица
        comparison = (
            sector_g
            .select([
                'inn', 'region',
                'line_2110', 'line_2400', 'line_1600',
                'ros', 'roa', 'leverage', 'current_ratio'
            ])
            .with_columns([
                (pl.col('line_2110') / 1_000_000_000).alias('выручка_млрд'),
                (pl.col('line_2400') / 1_000_000_000).alias('прибыль_млрд'),
                (pl.col('line_1600') / 1_000_000_000).alias('активы_млрд'),
                (pl.col('ros') * 100).alias('ROS_%'),
                (pl.col('roa') * 100).alias('ROA_%'),
            ])
            .sort('выручка_млрд', descending=True)
            .head(15)
        )
        
        print("\nСравнительная таблица компаний секции G:")
        print(comparison.select([
            'inn', 'region', 'выручка_млрд', 'прибыль_млрд', 
            'ROS_%', 'ROA_%', 'leverage', 'current_ratio'
        ]))
        
        # Статистика по отрасли
        print("\nСредние показатели по отрасли:")
        sector_stats = (
            sector_g
            .select([
                pl.mean('line_2110').alias('средняя_выручка'),
                pl.mean('line_2400').alias('средняя_прибыль'),
                pl.mean('ros').alias('средняя_ROS'),
                pl.mean('roa').alias('средняя_ROA'),
                pl.mean('leverage').alias('средний_леверидж'),
            ])
            .with_columns([
                (pl.col('средняя_выручка') / 1_000_000_000).alias('выручка_млрд'),
                (pl.col('средняя_прибыль') / 1_000_000_000).alias('прибыль_млрд'),
                (pl.col('средняя_ROS') * 100).alias('ROS_%'),
                (pl.col('средняя_ROA') * 100).alias('ROA_%'),
            ])
        )
        print(sector_stats.select(['выручка_млрд', 'прибыль_млрд', 'ROS_%', 'ROA_%', 'средний_леверидж']))
    else:
        print("Не найдено компаний в секции G с выручкой > 500 млн руб.")
else:
    print("Сначала выполните расчет финансовых показателей")


### Пример 5: Анализ динамики показателей за несколько лет

Сравнение финансовых показателей компаний за разные годы для анализа трендов.


In [ ]:
# Пример 5: Анализ динамики показателей за несколько лет
# Для этого примера нужно загрузить данные за несколько лет

# Загружаем данные за 2022-2024 годы с расчетом показателей
lf_multiyear = load_rfsd_years_polars(
    [2022, 2023, 2024],
    columns=key_columns
)

# Применяем фильтры
lf_multiyear_filtered = lf_multiyear.filter(
    (pl.col('filed') == 1) & 
    (pl.col('line_2110') > 1_000_000_000)
)

# Рассчитываем показатели на уровне LazyFrame
lf_multiyear_with_ratios = calculate_financial_ratios_polars(lf_multiyear_filtered)

# Фильтруем валидные значения
lf_multiyear_valid = lf_multiyear_with_ratios.filter(
    (pl.col('ros').is_finite()) &
    (pl.col('roa').is_finite())
)

# Загружаем данные (может занять время!)
# Раскомментируйте следующую строку для выполнения:
# df_multiyear = lf_multiyear_valid.collect()

# Анализ динамики по годам
if 'df_multiyear' in locals():
    print("Динамика средних показателей по годам:")
    yearly_stats = (
        df_multiyear
        .group_by('year')
        .agg([
            pl.mean('line_2110').alias('средняя_выручка'),
            pl.mean('line_2400').alias('средняя_прибыль'),
            pl.mean('ros').alias('средняя_ROS'),
            pl.mean('roa').alias('средняя_ROA'),
            pl.count().alias('количество_компаний')
        ])
        .sort('year')
        .with_columns([
            (pl.col('средняя_выручка') / 1_000_000_000).alias('выручка_млрд'),
            (pl.col('средняя_прибыль') / 1_000_000_000).alias('прибыль_млрд'),
            (pl.col('средняя_ROS') * 100).alias('ROS_%'),
            (pl.col('средняя_ROA') * 100).alias('ROA_%'),
        ])
    )
    print(yearly_stats.select(['year', 'количество_компаний', 'выручка_млрд', 'прибыль_млрд', 'ROS_%', 'ROA_%']))
    
    # Визуализация динамики
    import matplotlib.pyplot as plt
    
    yearly_pd = yearly_stats.select(['year', 'ROS_%', 'ROA_%']).to_pandas()
    
    plt.figure(figsize=(12, 6))
    plt.plot(yearly_pd['year'], yearly_pd['ROS_%'], marker='o', label='ROS (%)', linewidth=2)
    plt.plot(yearly_pd['year'], yearly_pd['ROA_%'], marker='s', label='ROA (%)', linewidth=2)
    plt.xlabel('Год')
    plt.ylabel('Показатель (%)')
    plt.title('Динамика рентабельности по годам')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Для анализа динамики раскомментируйте строку с df_multiyear = lf_multiyear_valid.collect()")
    print("Это может занять некоторое время из-за большого объема данных")


## Практические задания

### Задание 1: Изучение структуры данных

1. Загрузите данные RFSD за 2024 год
2. Изучите доступные переменные и их названия
3. Определите ключевые переменные для анализа:
   - Идентификаторы компаний (ИНН, название)
   - Финансовые показатели (выручка, активы, прибыль)
   - Классификаторы (регион, отрасль, ОКВЭД)

### Задание 2: Фильтрация данных

1. Создайте подвыборку компаний по критериям:
   - Размер компании (например, выручка > 1 млрд руб.)
   - Регион регистрации
   - Отрасль деятельности
2. Обоснуйте выбор критериев

### Задание 3: Расчет финансовых показателей

1. Рассчитайте ключевые финансовые мультипликаторы:
   - Рентабельность продаж
   - Рентабельность активов
   - Долговая нагрузка
   - Ликвидность
2. Найдите компании с экстремальными значениями показателей

### Задание 4: Сравнительный анализ

1. Выберите 3-5 компаний из одной отрасли
2. Сравните их финансовые показатели
3. Определите лидера и аутсайдера отрасли
4. Обоснуйте выводы

### Задание 5: Визуализация

1. Постройте графики распределения финансовых показателей
2. Создайте scatter plot для анализа взаимосвязи между показателями
3. Визуализируйте сравнительный анализ компаний


## Альтернативный способ загрузки: Hugging Face Datasets

Для работы с полным датасетом можно также использовать библиотеку `datasets` от Hugging Face, которая оптимизирована для больших данных:

```python
from datasets import load_dataset

# Загрузка через Hugging Face (автоматически использует эффективные методы)
# dataset = load_dataset('irlspbru/RFSD', split='train')

# Фильтрация и выбор столбцов на уровне загрузки
# dataset = load_dataset(
#     'irlspbru/RFSD', 
#     split='train',
#     columns=key_columns
# )
```

**Преимущества Hugging Face Datasets:**
- Автоматическая оптимизация памяти
- Поддержка streaming для очень больших данных
- Кэширование загруженных данных
- Интеграция с другими инструментами ML

## Дополнительные ресурсы

- **Документация RFSD:** https://github.com/irlcode/RFSD
- **Статья о RFSD:** https://doi.org/10.1038/s41597-025-05150-1
- **Hugging Face:** https://huggingface.co/datasets/irlspbru/RFSD
- **Документация Polars:** https://pola-rs.github.io/polars/

## Рекомендации по работе с большими данными

1. **Используйте Lazy API Polars:** Всегда начинайте с `scan_parquet()` и применяйте фильтры до `.collect()`

2. **Выбирайте только нужные столбцы:** В RFSD более 200 переменных, загружайте только те, что нужны для анализа

3. **Применяйте фильтры на уровне чтения:** Polars может фильтровать данные при чтении Parquet файлов

4. **Работайте с подвыборками:** Для интерактивного анализа загружайте только отфильтрованные данные

5. **Используйте партиционирование:** Данные организованы по годам - загружайте только нужные периоды

6. **Конвертируйте в pandas только для визуализации:** Polars быстрее для вычислений, pandas удобнее для графиков


In [26]:
from datasets.utils import get_cache_directory

print(get_cache_directory()) # узнать кэш 

# setx HF_HOME "D:\hf_cache"


ImportError: The pyarrow installation is not built with support for the Parquet file format (DLL load failed while importing _parquet: Не найдена указанная процедура.)